In [32]:
!pip install pytorch_lightning #torch==1.3.1
!pip install transformers
!pip3 -q install emoji


In [33]:
%matplotlib inline

import json
import logging
from pathlib import Path
import random
import tarfile
import tempfile
import warnings

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
# import pandas_path  # Path style access for pandas
from tqdm import tqdm

import torch                    
import torchvision
import fasttext
from torchvision import datasets, models, transforms
import torch.nn as nn
from torch.nn import functional as F
import torch.optim as optim

In [34]:
import ast
import pandas as pd

def load_data_lists(path):
    data_points_lists = []
    with open(path, encoding='utf-8') as f:
        lines = f.readlines()

        for line in lines:
            try:
                data_points_lists.append(ast.literal_eval(line))
            except:
                # Ignore lines with errors
                pass

    print('Found {} lines in "{}".'.format(len(lines), path))
    print('Successfully loaded {} data points from "{}".'.format(len(data_points_lists), path))
    
    return data_points_lists

COLUMN_NAMES = ['ID', 'Text', 'Sarcastic']

def construct_df(data_points_lists, column_names=COLUMN_NAMES):
    df = pd.DataFrame(data_points_lists, columns=column_names)
    df['ID'] = pd.to_numeric(df['ID'])
    df['Sarcastic'] = df['Sarcastic'].astype('bool')
    
    return df

In [35]:
df_train = construct_df(load_data_lists('/Users/albert/Documents/CSE8813_DLT/Sarcasm-Detection-hierarchical_fusion_model/text_data/train.txt'))

print(df_train.shape)
print(df_train.info())

display(df_train.head())

Found 29040 lines in "/Users/albert/Documents/CSE8813_DLT/Sarcasm-Detection-hierarchical_fusion_model/text_data/train.txt".
Successfully loaded 29040 data points from "/Users/albert/Documents/CSE8813_DLT/Sarcasm-Detection-hierarchical_fusion_model/text_data/train.txt".
(29040, 3)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29040 entries, 0 to 29039
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   ID         29040 non-null  int64 
 1   Text       29040 non-null  object
 2   Sarcastic  29040 non-null  bool  
dtypes: bool(1), int64(1), object(1)
memory usage: 482.2+ KB
None


,ID,Text,Sarcastic
0,910308516510011393,most # funny quotes : 21 snarky and # funny qu...,True
1,725333760762363905,spurs # creativethinking ! <url>,True
2,840006160660983809,<user> thanks for showing up for our appointme...,True
3,854334602516733952,only a hardcore fan of sir jonny sins will get...,True
4,908913372199915520,haha . # lol,True


In [36]:
df_val = construct_df(load_data_lists('/Users/albert/Documents/CSE8813_DLT/Sarcasm-Detection-hierarchical_fusion_model/text_data/valid.txt'), COLUMN_NAMES + ['Sarcastic_human'])
print(df_val.shape)
print(df_val.info())

display(df_val.head())

Found 2410 lines in "/Users/albert/Documents/CSE8813_DLT/Sarcasm-Detection-hierarchical_fusion_model/text_data/valid.txt".
Successfully loaded 2410 data points from "/Users/albert/Documents/CSE8813_DLT/Sarcasm-Detection-hierarchical_fusion_model/text_data/valid.txt".
(2410, 4)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2410 entries, 0 to 2409
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   ID               2410 non-null   int64 
 1   Text             2410 non-null   object
 2   Sarcastic        2410 non-null   bool  
 3   Sarcastic_human  2410 non-null   int64 
dtypes: bool(1), int64(2), object(1)
memory usage: 59.0+ KB
None


,ID,Text,Sarcastic,Sarcastic_human
0,915657464401580032,whew ... that extra <num> miles today to the g...,True,1
1,854678856724340736,""" oh , good . now no one will know we 're here...",True,1
2,904892917277274112,how much of it you think is true ? has this be...,True,1
3,855466461296504832,<user> finally found proof that the earth is f...,True,1
4,927373534652805120,many ways to overcome tension & fear but nothi...,True,1


In [37]:
df_test = construct_df(load_data_lists('/Users/albert/Documents/CSE8813_DLT/Sarcasm-Detection-hierarchical_fusion_model/text_data/test.txt'), COLUMN_NAMES + ['Sarcastic_human'])
print(df_test.shape)
print(df_test.info())

display(df_test.head())

Found 2409 lines in "/Users/albert/Documents/CSE8813_DLT/Sarcasm-Detection-hierarchical_fusion_model/text_data/test.txt".
Successfully loaded 2409 data points from "/Users/albert/Documents/CSE8813_DLT/Sarcasm-Detection-hierarchical_fusion_model/text_data/test.txt".
(2409, 4)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2409 entries, 0 to 2408
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   ID               2409 non-null   int64 
 1   Text             2409 non-null   object
 2   Sarcastic        2409 non-null   bool  
 3   Sarcastic_human  2409 non-null   int64 
dtypes: bool(1), int64(2), object(1)
memory usage: 58.9+ KB
None


,ID,Text,Sarcastic,Sarcastic_human
0,862902619928506372,i am guessing # netflix no longer lets you gra...,True,1
1,892551658487631873,it 's the insensitive strikeouts at suntrust p...,True,1
2,853143461360480256,"following the path of the river calder , so .....",True,1
3,918423568823840768,# westernsahara # authority has no lessons 2ge...,True,1
4,731617467718610944,hey <user> great sale !,True,1


In [38]:
!gdown --id 1dJERrVlp7DlNSXk-uvbbG6Rv7uvqTOKd


Downloading...
From: https://drive.google.com/uc?id=1dJERrVlp7DlNSXk-uvbbG6Rv7uvqTOKd
To: /Users/albert/Documents/CSE8813_DLT/dataset_image.z01
  8%|██▉                                    | 55.6M/734M [00:00<00:08, 79.4MB/s]^C
Traceback (most recent call last):
  File "/Users/albert/opt/anaconda3/envs/DLT_project2/bin/gdown", line 10, in <module>
    sys.exit(main())
  File "/Users/albert/opt/anaconda3/envs/DLT_project2/lib/python3.8/site-packages/gdown/cli.py", line 145, in main
    filename = download(
  File "/Users/albert/opt/anaconda3/envs/DLT_project2/lib/python3.8/site-packages/gdown/download.py", line 241, in download
    for chunk in res.iter_content(chunk_size=CHUNK_SIZE):
  File "/Users/albert/opt/anaconda3/envs/DLT_project2/lib/python3.8/site-packages/requests/models.py", line 758, in generate
    for chunk in self.raw.stream(chunk_size, decode_content=True):
  File "/Users/albert/opt/anaconda3/envs/DLT_project2/lib/python3.8/site-packages/urllib3/response.py", line 576, in

In [9]:
!gdown --id 1pODuKC4gP6-QDQonG8XTqI8w8ds68mE3


Downloading...
From: https://drive.google.com/uc?id=1pODuKC4gP6-QDQonG8XTqI8w8ds68mE3
To: /Users/albert/Documents/CSE8813_DLT/dataset_image.z02
100%|████████████████████████████████████████| 734M/734M [00:09<00:00, 75.8MB/s]


In [10]:
!gdown --id 1mK0Nf-jv_h2bgHUCRM4_EsdTiiitZ_Uj


Downloading...
From: https://drive.google.com/uc?id=1mK0Nf-jv_h2bgHUCRM4_EsdTiiitZ_Uj
To: /Users/albert/Documents/CSE8813_DLT/dataset_image.z03
100%|████████████████████████████████████████| 734M/734M [00:09<00:00, 77.6MB/s]


In [11]:
!gdown --id 1AOWzlOz5hmdO39dEmzhQ4z_nabgzi7Tu


Downloading...
From: https://drive.google.com/uc?id=1AOWzlOz5hmdO39dEmzhQ4z_nabgzi7Tu
To: /Users/albert/Documents/CSE8813_DLT/dataset_image.zip
100%|████████████████████████████████████████| 460M/460M [00:05<00:00, 77.8MB/s]


In [12]:
!zip -FF dataset_image.zip --out dataset_image_full.zip
!unzip -FF dataset_image_full.zip -d /content

Fix archive (-FF) - salvage what can
 Found end record (EOCDR) - says expect 4 splits
Scanning for entries...
  Found spanning marker - expected as this is split (multi-disk) archive...
 copying: dataset_image/  (0 bytes)
 copying: dataset_image/682716753374351360.jpg  (91933 bytes)
 copying: dataset_image/682721949072625664.jpg  (11410 bytes)
 copying: dataset_image/682722242111774720.jpg  (55338 bytes)
 copying: dataset_image/682727470433308672.jpg  (20498 bytes)
 copying: dataset_image/682751920956641280.jpg  (47665 bytes)
 copying: dataset_image/682762265041321989.jpg  (64143 bytes)
 copying: dataset_image/682773510196822016.jpg  (111873 bytes)
 copying: dataset_image/682783114343518208.jpg  (97818 bytes)
 copying: dataset_image/682784347305410560.jpg  (47626 bytes)
 copying: dataset_image/682836685323956224.jpg  (18549 bytes)
 copying: dataset_image/682864661813981184.jpg  (167523 bytes)
 copying: dataset_image/682883715681390593.jpg  (109367 bytes)
 copying: dataset_image/6829060

 copying: dataset_image/685941579341664257.jpg  (121564 bytes)
 copying: dataset_image/685982606496776192.jpg  (123558 bytes)
 copying: dataset_image/685996770829168640.jpg  (115359 bytes)
 copying: dataset_image/685999243874676736.jpg  (88252 bytes)
 copying: dataset_image/686013652021809152.jpg  (110422 bytes)
 copying: dataset_image/686029041900507136.jpg  (142632 bytes)
 copying: dataset_image/686050853120167936.jpg  (160007 bytes)
 copying: dataset_image/686074568759357441.jpg  (72882 bytes)
 copying: dataset_image/686096922868867072.jpg  (218620 bytes)
 copying: dataset_image/686126823592177664.jpg  (24638 bytes)
 copying: dataset_image/686222316431257600.jpg  (26645 bytes)
 copying: dataset_image/686244213932048385.jpg  (2146 bytes)
 copying: dataset_image/686249757103271937.jpg  (86242 bytes)
 copying: dataset_image/686253463446138880.jpg  (44714 bytes)
 copying: dataset_image/686253993769734145.jpg  (34976 bytes)
 copying: dataset_image/686269922779873280.jpg  (19039 bytes)
 c

 copying: dataset_image/689600450018938880.jpg  (193289 bytes)
 copying: dataset_image/689637362058440705.jpg  (148708 bytes)
 copying: dataset_image/689667649878540289.jpg  (98590 bytes)
 copying: dataset_image/689681644987551744.jpg  (41686 bytes)
 copying: dataset_image/689691065503326208.jpg  (99333 bytes)
 copying: dataset_image/689704577847918592.jpg  (150083 bytes)
 copying: dataset_image/689751315505926144.jpg  (88386 bytes)
 copying: dataset_image/689752136435929088.jpg  (139234 bytes)
 copying: dataset_image/689782673112367104.jpg  (71289 bytes)
 copying: dataset_image/689796494157832193.jpg  (32108 bytes)
 copying: dataset_image/689796910463467520.jpg  (154981 bytes)
 copying: dataset_image/689816370658349057.jpg  (44324 bytes)
 copying: dataset_image/689825646760042497.jpg  (88669 bytes)
 copying: dataset_image/689841858198007809.jpg  (139986 bytes)
 copying: dataset_image/689845434211143680.jpg  (71462 bytes)
 copying: dataset_image/689845804610158592.jpg  (91876 bytes)
 c

 copying: dataset_image/692150765788463105.jpg  (77625 bytes)
 copying: dataset_image/692203022550745088.jpg  (109196 bytes)
 copying: dataset_image/692220407160672256.jpg  (56088 bytes)
 copying: dataset_image/692234893082935297.jpg  (69890 bytes)
 copying: dataset_image/692285316078669825.jpg  (82286 bytes)
 copying: dataset_image/692304634870763521.jpg  (34781 bytes)
 copying: dataset_image/692319239944982528.jpg  (92581 bytes)
 copying: dataset_image/692330929432432640.jpg  (66487 bytes)
 copying: dataset_image/692363616058617857.jpg  (41395 bytes)
 copying: dataset_image/692377226956902400.jpg  (163089 bytes)
 copying: dataset_image/692387490775306240.jpg  (112379 bytes)
 copying: dataset_image/692419850942177280.jpg  (64038 bytes)
 copying: dataset_image/692439296465313792.jpg  (83837 bytes)
 copying: dataset_image/692462167568846849.jpg  (196915 bytes)
 copying: dataset_image/692473402402152448.jpg  (36906 bytes)
 copying: dataset_image/692490360384307204.jpg  (22209 bytes)
 cop

 copying: dataset_image/696863713723179008.jpg  (171187 bytes)
 copying: dataset_image/696900576014049280.jpg  (113106 bytes)
 copying: dataset_image/697013421414289408.jpg  (78798 bytes)
 copying: dataset_image/697013893822939136.jpg  (151907 bytes)
 copying: dataset_image/697021286783021056.jpg  (168597 bytes)
 copying: dataset_image/697030480995164160.jpg  (38093 bytes)
 copying: dataset_image/697060889720131584.jpg  (90330 bytes)
 copying: dataset_image/697115545720610816.jpg  (269423 bytes)
 copying: dataset_image/697122270796505088.jpg  (55164 bytes)
 copying: dataset_image/697142113302085633.jpg  (124961 bytes)
 copying: dataset_image/697142267849674752.jpg  (97515 bytes)
 copying: dataset_image/697161140607000576.jpg  (32919 bytes)
 copying: dataset_image/697167701442215936.jpg  (40638 bytes)
 copying: dataset_image/697173614664134656.jpg  (38202 bytes)
 copying: dataset_image/697199578731409409.jpg  (108060 bytes)
 copying: dataset_image/697201021706067968.jpg  (114028 bytes)


...

 copying: dataset_image/926577438150107137.jpg  (152817 bytes)
 copying: dataset_image/926589171606663169.jpg  (15129 bytes)
 copying: dataset_image/926614628850905088.jpg  (24883 bytes)
 copying: dataset_image/926638250126659584.jpg  (32310 bytes)
 copying: dataset_image/926674894833807360.jpg  (30208 bytes)
 copying: dataset_image/926684079021465600.jpg  (33584 bytes)
 copying: dataset_image/926689981669654528.jpg  (233626 bytes)
 copying: dataset_image/926700025010315265.jpg  (85521 bytes)
 copying: dataset_image/926702987929575424.jpg  (161521 bytes)
 copying: dataset_image/926742292433473537.jpg  (60677 bytes)
 copying: dataset_image/926755019361157120.jpg  (308958 bytes)
 copying: dataset_image/926769947908358147.jpg  (104362 bytes)
 copying: dataset_image/926772943564128256.jpg  (83191 bytes)
 copying: dataset_image/926819047643144192.jpg  (42888 bytes)
 copying: dataset_image/926833664272433152.jpg  (21778 bytes)
 copying: dataset_image/926852642201989120.jpg  (14682 bytes)
 co

 copying: dataset_image/931031750100127749.jpg  (196300 bytes)
 copying: dataset_image/931056706448216064.jpg  (49089 bytes)
 copying: dataset_image/931062242946310149.jpg  (148260 bytes)
 copying: dataset_image/931164042491699200.jpg  (14091 bytes)
 copying: dataset_image/931183048313442305.jpg  (68586 bytes)
 copying: dataset_image/931209536077225984.jpg  (17518 bytes)
 copying: dataset_image/931218695111704576.jpg  (112340 bytes)
 copying: dataset_image/931247770136207362.jpg  (40397 bytes)
 copying: dataset_image/931253047468191746.jpg  (135051 bytes)
 copying: dataset_image/931273486965923845.jpg  (124323 bytes)
 copying: dataset_image/931284151734538240.jpg  (64681 bytes)
 copying: dataset_image/931285870207832064.jpg  (45022 bytes)
 copying: dataset_image/931310930121469952.jpg  (205017 bytes)
 copying: dataset_image/931316232904638464.jpg  (112790 bytes)
 copying: dataset_image/931343690701918208.jpg  (199937 bytes)
 copying: dataset_image/931358326922780672.jpg  (86548 bytes)


 copying: dataset_image/935224883683790848.jpg  (116144 bytes)
 copying: dataset_image/935229281046708225.jpg  (108539 bytes)
 copying: dataset_image/935237658766446592.jpg  (152353 bytes)
 copying: dataset_image/935243492493807616.jpg  (176841 bytes)
 copying: dataset_image/935320976614715392.jpg  (207119 bytes)
 copying: dataset_image/935324513226633216.jpg  (74978 bytes)
 copying: dataset_image/935327532580302848.jpg  (21104 bytes)
 copying: dataset_image/935341666248151042.jpg  (57531 bytes)
 copying: dataset_image/935525638483652609.jpg  (91902 bytes)
 copying: dataset_image/935527705671434244.jpg  (59108 bytes)
 copying: dataset_image/935568867908378625.jpg  (85165 bytes)
 copying: dataset_image/935576631300841473.jpg  (105735 bytes)
 copying: dataset_image/935596637820370944.jpg  (58527 bytes)
 copying: dataset_image/935626998445170688.jpg  (79719 bytes)
 copying: dataset_image/935627156155088896.jpg  (17223 bytes)
 copying: dataset_image/935658641784532992.jpg  (50042 bytes)
 c

 copying: dataset_image/938800275946639360.jpg  (64252 bytes)
 copying: dataset_image/938813110806351873.jpg  (135774 bytes)
 copying: dataset_image/938819144396423168.jpg  (90298 bytes)
 copying: dataset_image/938822920339623937.jpg  (64578 bytes)
 copying: dataset_image/938830482170671105.jpg  (86227 bytes)
 copying: dataset_image/938838173291081729.jpg  (55497 bytes)
 copying: dataset_image/938854362490605569.jpg  (55143 bytes)
 copying: dataset_image/938858716358676480.jpg  (23445 bytes)
 copying: dataset_image/938868490189946880.jpg  (113234 bytes)
 copying: dataset_image/938873673645047808.jpg  (11421 bytes)
 copying: dataset_image/938888425100529666.jpg  (44139 bytes)
 copying: dataset_image/938897664111935499.jpg  (111104 bytes)
 copying: dataset_image/938899544682704897.jpg  (111180 bytes)
 copying: dataset_image/938901693768626176.jpg  (46741 bytes)
 copying: dataset_image/938950575064080384.jpg  (91178 bytes)
 copying: dataset_image/939010628236165120.jpg  (56339 bytes)
 cop

EOCDR found ( 4 459858515)...
UnZip 6.00 of 20 April 2009, by Info-ZIP.  Maintained by C. Spieler.  Send
bug reports using http://www.info-zip.org/zip-bug.html; see README for details.

Usage: unzip [-Z] [-opts[modifiers]] file[.zip] [list] [-x xlist] [-d exdir]
  Default action is to extract files in list, except those in xlist, to exdir;
  file[.zip] may be a wildcard.  -Z => ZipInfo mode ("unzip -Z" for usage).

  -p  extract files to pipe, no messages     -l  list files (short format)
  -f  freshen existing files, create none    -t  test compressed archive data
  -u  update files, create if necessary      -z  display archive comment only
  -v  list verbosely/show version info       -T  timestamp archive to latest
  -x  exclude files that follow (in xlist)   -d  extract files into exdir
modifiers:
  -n  never overwrite existing files         -q  quiet mode (-qq => quieter)
  -o  overwrite files WITHOUT prompting      -a  auto-convert any text files
  -j  junk paths (do not make dire

In [39]:
# Create subdirectories
import os
import shutil

train_dir = '/Users/albert/Documents/CSE8813_DLT/Sarcasm-Detection-hierarchical_fusion_model/image_data/train/'
val_dir = '/Users/albert/Documents/CSE8813_DLT/Sarcasm-Detection-hierarchical_fusion_model/image_data/val/'
test_dir = '/Users/albert/Documents/CSE8813_DLT/Sarcasm-Detection-hierarchical_fusion_model/image_data/test/'

os.mkdir(train_dir)
os.mkdir(val_dir)
os.mkdir(test_dir)

In [40]:
missing_ids = []

for id in df_train['ID']:
    img_path = '/Users/albert/Documents/CSE8813_DLT/Sarcasm-Detection-hierarchical_fusion_model/image_data/{}.jpg'.format(id)
    #img_path = 'dataset_image/{}.jpg'.format(id)
    if os.path.isfile(img_path):
        #shutil.move(img_path, '/Users/albert/Documents/CSE8813_DLT/Sarcasm-Detection-hierarchical_fusion_model/image_data/train/{}.jpg'.format(id))

      shutil.move(img_path, '/Users/albert/Documents/CSE8813_DLT/Sarcasm-Detection-hierarchical_fusion_model/image_data/train/{}.jpg'.format(id))
    else:
        missing_ids.append(id)

In [41]:
len(missing_ids)


9224

In [42]:
df_train = df_train[~df_train['ID'].isin(missing_ids)]
df_train.shape

(19816, 3)

In [43]:
for id in df_val['ID']:
    img_path = '/Users/albert/Documents/CSE8813_DLT/Sarcasm-Detection-hierarchical_fusion_model/image_data/{}.jpg'.format(id)
    if os.path.isfile(img_path):
      shutil.move(img_path, '/Users/albert/Documents/CSE8813_DLT/Sarcasm-Detection-hierarchical_fusion_model/image_data/val/{}.jpg'.format(id))

In [44]:
for id in df_test['ID']:
    img_path = '/Users/albert/Documents/CSE8813_DLT/Sarcasm-Detection-hierarchical_fusion_model/image_data/{}.jpg'.format(id)
    if os.path.isfile(img_path):
      shutil.move(img_path, '/Users/albert/Documents/CSE8813_DLT/Sarcasm-Detection-hierarchical_fusion_model/image_data/test/{}.jpg'.format(id))

In [45]:
os.mkdir('/Users/albert/Documents/CSE8813_DLT/Sarcasm-Detection-hierarchical_fusion_model/image_data/train/sarc/')
os.mkdir('/Users/albert/Documents/CSE8813_DLT/Sarcasm-Detection-hierarchical_fusion_model/image_data/train/non-sarc/')
os.mkdir('/Users/albert/Documents/CSE8813_DLT/Sarcasm-Detection-hierarchical_fusion_model/image_data/val/sarc/')
os.mkdir('/Users/albert/Documents/CSE8813_DLT/Sarcasm-Detection-hierarchical_fusion_model/image_data/val/non-sarc/')
os.mkdir('/Users/albert/Documents/CSE8813_DLT/Sarcasm-Detection-hierarchical_fusion_model/image_data/test/sarc/')
os.mkdir('/Users/albert/Documents/CSE8813_DLT/Sarcasm-Detection-hierarchical_fusion_model/image_data/test/non-sarc/')

In [46]:
class_foldername = {
    True: 'sarc',
    False: 'non-sarc'
}

In [47]:
# Move images to subfolders
for id,label in zip(df_train['ID'], df_train['Sarcastic']):
    shutil.move('/Users/albert/Documents/CSE8813_DLT/Sarcasm-Detection-hierarchical_fusion_model/image_data/train/{}.jpg'.format(id), '/Users/albert/Documents/CSE8813_DLT/Sarcasm-Detection-hierarchical_fusion_model/image_data/train/{}/{}.jpg'.format(class_foldername[label], id))

In [48]:
# Move images to subfolders
for id,label in zip(df_val['ID'], df_val['Sarcastic']):
    shutil.move('/Users/albert/Documents/CSE8813_DLT/Sarcasm-Detection-hierarchical_fusion_model/image_data/val/{}.jpg'.format(id), '/Users/albert/Documents/CSE8813_DLT/Sarcasm-Detection-hierarchical_fusion_model/image_data/val/{}/{}.jpg'.format(class_foldername[label], id))


In [49]:
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])

data_transforms = {
    'train':
    transforms.Compose([
        transforms.Resize((224,224)),
        transforms.RandomAffine(0, shear=10, scale=(0.8,1.2)),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        normalize
    ]),
    'validation':
    transforms.Compose([
        transforms.Resize((224,224)),
        transforms.ToTensor(),
        normalize
    ]),
}

image_datasets = {
    'train': 
    datasets.ImageFolder('/Users/albert/Documents/CSE8813_DLT/Sarcasm-Detection-hierarchical_fusion_model/image_data/train', data_transforms['train']),
    'validation': 
    datasets.ImageFolder('/Users/albert/Documents/CSE8813_DLT/Sarcasm-Detection-hierarchical_fusion_model/image_data/val', data_transforms['validation'])
}

image_dataloaders = {
    'train':
    torch.utils.data.DataLoader(image_datasets['train'],
                                batch_size=64,
                                shuffle=True),
    'validation':
    torch.utils.data.DataLoader(image_datasets['validation'],
                                batch_size=64,
                                shuffle=False)
}

In [50]:
# dataloader for txt data
from transformers import AutoModel, AutoTokenizer

from sklearn.utils import shuffle
from transformers import (get_linear_schedule_with_warmup,AdamW,AutoModel, AutoTokenizer,
                            AutoModelForSequenceClassification)
from torch.utils.data import (TensorDataset,DataLoader,
                             RandomSampler, SequentialSampler, Dataset)


train_df = shuffle(df_train, random_state=42)
valid_df = shuffle(df_val, random_state=42)
test_df = shuffle(df_test, random_state=42)


In [51]:
bertweet = AutoModel.from_pretrained("vinai/bertweet-base")
tokenizer = AutoTokenizer.from_pretrained("vinai/bertweet-base", use_fast=False)

text_model = AutoModelForSequenceClassification.from_pretrained(
    "vinai/bertweet-base",
        num_labels = 2,  
        output_attentions = False, 
        output_hidden_states = True,
    )

Some weights of the model checkpoint at vinai/bertweet-base were not used when initializing RobertaModel: ['lm_head.layer_norm.bias', 'lm_head.dense.bias', 'lm_head.decoder.bias', 'lm_head.layer_norm.weight', 'lm_head.bias', 'lm_head.dense.weight', 'lm_head.decoder.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Some weights of the model checkpoint at vinai/bertweet-base were not used when initializing RobertaForSequenceClassification: ['lm_head.layer_norm.b

In [52]:
def bert_encode(df, tokenizer):
    input_ids = []
    attention_masks = []
    # print(df)
    for sent,obj in zip(df[['Text']].values,df["object"].values):
        sent = sent.item()
        #print("sent")
        #print(sent)
        #obj = obj.item()
        #print(obj)
        sent = obj+sent
        print("combined")
        print(sent)
        encoded_dict = tokenizer.encode_plus(
                            sent,                      
                            add_special_tokens = True, 
                            max_length = 128,           
                            pad_to_max_length = True,
                            truncation = True,
                            return_attention_mask = True,   
                            return_tensors = 'pt',    
                    )
           
        input_ids.append(encoded_dict['input_ids'])
        attention_masks.append(encoded_dict['attention_mask'])
    input_ids = torch.cat(input_ids, dim=0)
    attention_masks = torch.cat(attention_masks, dim=0)

    inputs = {
    'input_word_ids': input_ids,
    'input_mask': attention_masks}

    return inputs

In [53]:
from torch.utils.data import (TensorDataset,DataLoader,
                             RandomSampler, SequentialSampler, Dataset)

def prepare_dataloaders(train_df,test_df,batch_size=64):
    # Load the AutoTokenizer with a normalization mode if the input Tweet is raw
    
    tokenizer = AutoTokenizer.from_pretrained("vinai/bertweet-base", use_fast=False, normalization=True)
    
    tweet_train = bert_encode(train_df, tokenizer)
    tweet_train_labels = train_df['Sarcastic'].astype(int)
    
    tweet_test = bert_encode(test_df, tokenizer)

    input_ids, attention_masks = tweet_train.values()
    labels = torch.tensor(tweet_train_labels.values)
    train_dataset = TensorDataset(input_ids, attention_masks, labels)

    
    input_ids, attention_masks = tweet_test.values()
    test_dataset = TensorDataset(input_ids, attention_masks)

    
    train_dataloader = DataLoader(
                train_dataset,
                sampler = RandomSampler(train_dataset), 
                batch_size = batch_size 
            )


    test_dataloader = DataLoader(
                test_dataset, 
                sampler = SequentialSampler(test_dataset), 
                batch_size = batch_size
            )
    return train_dataloader, test_dataloader

In [54]:
#text_train_dataloader,text_test_dataloader = prepare_dataloaders(train_df, test_df)
#_,text_val_dataloader = prepare_dataloaders(train_df, valid_df)


In [55]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cpu')

In [ ]:
import csv
#print(df_train)



with open('/Users/albert/Documents/CSE8813_DLT/object_image2.csv') as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    for row in csv_reader:
        ID = row[0]
        obj = row[1]
        
        str_obj = obj.replace("[","")
        str_obj = str_obj.replace("]","")
        train_df.loc[df_train["ID"] == int(row[0]),"object"] = str_obj
        

In [58]:

#print(df_train["object"])
print(train_df.isna().sum())
train_df = train_df.fillna(' ')
print(train_df["object"])

ID           0
Text         0
Sarcastic    0
object       0
dtype: int64
2                 'tvmonitor'
4              'zebra', 'cow'
5                     'train'
6                            
7                            
                 ...         
29034                'person'
29035                        
29036    'person', 'backpack'
29037                'person'
29039                        
Name: object, Length: 19816, dtype: object


In [59]:
 text_train_dataloader,text_test_dataloader = prepare_dataloaders(train_df, test_df)
_,text_val_dataloader = prepare_dataloaders(train_df, valid_df)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


KeyError: 'object'

In [63]:
def prepare_model(model_class="vinai/bertweet-base",num_classes=2,model_to_load=None,total_steps=-1):
    
    #model = AutoModel.from_pretrained(model_class)

    model = AutoModelForSequenceClassification.from_pretrained(
        model_class,
        num_labels = num_classes,  
        output_attentions = False, 
        output_hidden_states = True,
    )

    optimizer = AdamW(model.parameters(),
                    lr = 5e-5,
                    eps = 1e-8
                    )
    scheduler = get_linear_schedule_with_warmup(optimizer, 
                                                num_warmup_steps = 0, 
                                                num_training_steps = total_steps)

    if model_to_load is not None:
        try:
            model.roberta.load_state_dict(torch.load(model_to_load))
            print("LOADED MODEL")
        except:
            pass
    return model, optimizer, scheduler

In [221]:
count = 0
for batch in text_train_dataloader:
    count+=1
    print(batch[0])
    if count==5:
        break

tensor([[    0,   465, 18451,  ...,     1,     1,     1],
        [    0,  8454,     9,  ...,     1,     1,     1],
        [    0, 14249,  5163,  ...,     1,     1,     1],
        ...,
        [    0,   191,   213,  ...,     1,     1,     1],
        [    0,    11, 14291,  ...,     1,     1,     1],
        [    0,  8215,   288,  ...,     1,     1,     1]])
tensor([[    0,     6, 11684,  ...,     1,     1,     1],
        [    0, 16708,   465,  ...,     1,     1,     1],
        [    0,   465, 43871,  ...,     1,     1,     1],
        ...,
        [    0,     6, 10426,  ...,     1,     1,     1],
        [    0,    33,    17,  ...,     1,     1,     1],
        [    0,     6,  5206,  ...,     1,     1,     1]])
tensor([[   0, 1832,    6,  ...,    1,    1,    1],
        [   0,   37,  863,  ...,    1,    1,    1],
        [   0,    6, 1340,  ...,    1,    1,    1],
        ...,
        [   0,  618, 7186,  ...,    1,    1,    1],
        [   0, 8454,    9,  ...,    1,    1,    1],
   

In [60]:
class MultiModel(nn.Module):
    def __init__(self):
        super(MultiModel, self).__init__()
        #Resnet50 for image
        self.text_model, _, _ = prepare_model("vinai/bertweet-base" ,num_classes=2, model_to_load=None, total_steps = total_steps)
        self.linear_1 = nn.Linear(768,300)
        #self.text_model = AutoModelForSequenceClassification.from_pretrained(
        #model_class = "vinai/bertweet-base",
        #num_labels = 2,  
        #output_attentions = False, 
        #output_hidden_states = False,)
        
        self.image_model = torchvision.models.resnet50(
            pretrained=True
        )
        #resnet_output_feature_dim = 300
        #bert_output_feature_dim = 300
        #fusion_input_feature = 300+300 = 600
        self.image_model.fc = nn.Sequential(
               nn.Linear(2048, 128),
               nn.ReLU(inplace=True),
               nn.Linear(128, 300))
        
        
        # Combined
        
        #fusion_input_feature = 300+300 = 600
        #fusion_output_size = 128
        self.combined_fc1 = nn.Linear(600, 256)
        self.combined_fc2 = nn.Linear(256, 128)
        
        self.output_fc = nn.Linear(128, 2)
       
        #self.text_train_dataset = image_dataloaders["train"]
        #self.text_dev_dataset = image_dataloaders["validation"]
        #self.image_train_dataset = text_train_dataloader
        #self.image_dev_dataset = text_val_dataloader

    
    def forward(self, text,text_mask,image):
        text = text
        bert_out = self.text_model(text)
        #last_layer = bert_out.last_hidden_state[:,0,:]
        #last_layer = self.linear_1(last_layer)
        #print(len(bert_out))
        
        last_layer = bert_out.hidden_states[-1][:, 0, :]
        #print(last_layer.shape)
        image = image
        resnet_out = self.image_model(image)
        print("resnet output")
        print(resnet_out.shape)
        last_layer= self.linear_1(last_layer)
        combined_emb = torch.cat((resnet_out, last_layer), 1)
        print("combined_emb")
        print(combined_emb.shape)
        x_comb = F.relu(self.combined_fc1(combined_emb))
        x_comb = F.relu(self.combined_fc2(x_comb))
        out = torch.sigmoid(self.output_fc(x_comb))
        #logits = self.output_fc(x_comb)
        #pred = torch.nn.functional.softmax(logits)
        return out


In [108]:
#count = 0
#for batch, [image,label] in zip(text_train_dataloader,image_dataloaders["train"]):
#    count+=1
#    print("text input shape")
#    print(batch[0].shape)
#    print(batch[1].shape)
#    print(batch[2].shape)
#    print("image input shape")
#    print(image.shape)
#    print(label.shape)
#    if count==5:
#        break

text input shape
torch.Size([64, 128])
torch.Size([64, 128])
torch.Size([64])
image input shape
torch.Size([64, 3, 224, 224])
torch.Size([64])
text input shape
torch.Size([64, 128])
torch.Size([64, 128])
torch.Size([64])
image input shape
torch.Size([64, 3, 224, 224])
torch.Size([64])
text input shape
torch.Size([64, 128])
torch.Size([64, 128])
torch.Size([64])
image input shape
torch.Size([64, 3, 224, 224])
torch.Size([64])
text input shape
torch.Size([64, 128])
torch.Size([64, 128])
torch.Size([64])
image input shape
torch.Size([64, 3, 224, 224])
torch.Size([64])
text input shape
torch.Size([64, 128])
torch.Size([64, 128])
torch.Size([64])
image input shape
torch.Size([64, 3, 224, 224])
torch.Size([64])


In [61]:
def train_model(model, image_data_loaders,text_train_data_loader,text_val_data_loader, criterion, optimizer, num_epochs=3):
    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch+1, num_epochs))
        print('-' * 10)

        for phase in ['train', 'validation']:
            if phase == 'train':
                model.train()
            else:
                model.eval()

            running_loss = 0.0
            running_corrects = 0

        
            for batch,[image_inputs,image_labels] in zip(text_train_data_loader,image_data_loaders[phase]):
                text_inputs = batch[0]
                text_mask = batch[1]
                text_labels = batch[2]
                text_inputs = text_inputs.to(device)
                text_labels = text_labels.to(device)
                image_inputs = image_inputs.to(device)
                image_labels = image_labels.to(device)
                outputs = model(text_inputs,text_mask,image_inputs)
                
                loss = criterion(outputs, image_labels)
                if phase == 'train':
                    optimizer.zero_grad()
                    loss.backward()
                    optimizer.step()

                _, preds = torch.max(outputs, 1)
                running_loss += loss.item() * image_inputs.size(0)
                running_corrects += torch.sum(preds == image_labels.data)

            epoch_loss = running_loss / len(image_datasets[phase])
            epoch_acc = running_corrects.double() / len(image_datasets[phase])

            print('{} loss: {:.4f}, acc: {:.4f}'.format(phase,
                                                        epoch_loss,
                                                        epoch_acc))
    return model

In [62]:
epochs = 10
total_steps = len(train_df) * epochs
model = MultiModel().to(device)

criterion = nn.CrossEntropyLoss()
optimizer = AdamW(model.parameters(),
                    lr = 5e-5,
                    eps = 1e-8
                    )

#loss_func = nn.MSELoss()
#optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

NameError: name 'prepare_model' is not defined

In [ ]:
model_trained = train_model(model,image_dataloaders,text_train_dataloader,text_val_dataloader, criterion, optimizer, num_epochs=10)

Epoch 1/10
----------


In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

In [ ]:
torch.save(model, '/content/drive/MyDrive/Sarcasm-Detection-hierarchical_fusion_model/multimodal_10epoch.pt')
